In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [3]:
df2017 = pd.read_csv('/content/drive/MyDrive/prescription_2017.csv')

In [ ]:
df2017

In [ ]:
sido_list = [11, 26, 27, 28, 29, 30, 31, 36, 41, 42, 43, 44, 45, 46, 47, 48, 49]

# 기존 1개씩 돌리던 코드

In [4]:
df2017_41 = df2017.loc[ df2017['SIDO'] == 41]

In [9]:
# Order Number가 동일한 Item Name을 하나의 리스트로 묶음
# 이 때, apriori 알고리즘의 정의에 의해, 중복된 연관 규칙은 제거할 필요가 있음

df_tmp = df2017_41.groupby('KEY_SEQ')['5_GNL'].apply(set).apply(list).to_list()
print(df_tmp[:10])

[['2170A', '2228A', '6395A'], ['2471A', '1457A'], ['2680A', '1305A', '5321A', '1349A'], ['1808A', '1132A', '1014A', '2121A'], ['4893A', '1994A'], ['2429A', '4636A', '5220A', '1115A', '2222A', '4423A', '4389A', '4802A', '1009A', '5325A', '4301A', '6149A'], ['1305A', '1622C', '2209A', '5325A'], ['2718A', '1531A', '1252A', '1728A'], ['2133A', '1831A', '2209A', '4389A', '1018A'], ['2718A', '1525A']]


In [10]:
# nested list (장바구니)를 association rule을 찾기 위한 dataframe의 형태로 변경

te = TransactionEncoder()
te_ary = te.fit_transform(df_tmp) # df_tmp를 장바구니 형태로 변경 (fit과 transform을 동시에 진행)
print(te_ary)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [11]:
df = pd.DataFrame(te_ary, columns=te.columns_) # label 정보는 te.columns_에 저장되어 있으므로, 해당 값을 칼럼명으로 하는 데이터 프레임으로 변경.
df

,1001A,1003A,1005A,1006A,1007A,1009A,1013A,1013C,1014A,1015A,...,6652A,6653A,6656A,8001C,8002C,8005C,8009C,8010C,8011C,8012C
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349854,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2349855,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2349856,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2349857,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
df_scaled = df.transpose()
df_scaled

,0,1,2,3,4,5,6,7,8,9,...,2349849,2349850,2349851,2349852,2349853,2349854,2349855,2349856,2349857,2349858
1001A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1003A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1005A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1006A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1007A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8005C,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8009C,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8010C,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8011C,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
sparse_df = csr_matrix(df_scaled)
drug_sim = cosine_similarity(sparse_df)
print(drug_sim.shape)

(1441, 1441)


In [14]:
drug_sim = pd.DataFrame(drug_sim, index = df_scaled.index, columns = df_scaled.index)
drug_sim

,1001A,1003A,1005A,1006A,1007A,1009A,1013A,1013C,1014A,1015A,...,6652A,6653A,6656A,8001C,8002C,8005C,8009C,8010C,8011C,8012C
1001A,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
1003A,0.0,1.000000,0.0,0.001279,0.000220,0.004065,0.000158,0.0,0.001150,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
1005A,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000134,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
1006A,0.0,0.001279,0.0,1.000000,0.000000,0.000398,0.000258,0.0,0.001486,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000
1007A,0.0,0.000220,0.0,0.000000,1.000000,0.002700,0.010023,0.0,0.041855,0.000000,...,0.0,0.0,0.0,0.00668,0.0,0.0,0.000000,0.000000,0.0,0.000393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8005C,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,1.0,0.000000,0.000000,0.0,0.000000
8009C,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.019645,...,0.0,0.0,0.0,0.00000,0.0,0.0,1.000000,0.168946,0.0,0.000000
8010C,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.004746,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.168946,1.000000,0.0,0.000000
8011C,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,1.0,0.000000


In [ ]:
drug_sim['5293A'].sort_values(ascending=False)[:100]

# Loop

In [ ]:
#loop로 가능할지

for i in sido_list :
    df2017_new = df2017.loc[ df2017['SIDO'] == i]
    #globals()['df2017_{}'.format(i)] = df2017_new

    df_tmp = df2017_new.groupby('KEY_SEQ')['5_GNL'].apply(set).apply(list).to_list()

    te = TransactionEncoder()
    te_ary = te.fit_transform(df_tmp)

    df = pd.DataFrame(te_ary, columns=te.columns_)
    df_scaled = df.transpose()

    sparse_df = csr_matrix(df_scaled)
    drug_sim = cosine_similarity(sparse_df)

    drug_sim = pd.DataFrame(drug_sim, index = df_scaled.index, columns = df_scaled.index)

    drug_sim.to_csv('/content/drive/MyDrive/CF_2017_'+str(i)+'.csv', index=True)
    print(i, len(drug_sim))

11 1439
26 1294
27 1252
28 1292
29 1195
30 1214
31 1139
36 946
41 1441
42 1227
43 1229
44 1267
45 1252
46 1255
47 1280
48 1312
49 1054


In [ ]:
df_tmp = df2017.groupby('KEY_SEQ')['5_GNL'].apply(set).apply(list).to_list()

te = TransactionEncoder()
te_ary = te.fit_transform(df_tmp)

df = pd.DataFrame(te_ary, columns=te.columns_)
df_scaled = df.transpose()

sparse_df = csr_matrix(df_scaled)
drug_sim = cosine_similarity(sparse_df)

drug_sim = pd.DataFrame(drug_sim, index = df_scaled.index, columns = df_scaled.index)

drug_sim.to_csv('/content/drive/MyDrive/CF_2017.csv', index=False)
print(len(drug_sim))

1612
